In [10]:
#pandas and numpy for df manipulation
import pandas as pd
import numpy as np
import re

#visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

#counter
from collections import Counter

#standard nlp libraries for text processing
import nltk

#gensim will not be used in this notebook, but it is a very popular and useful library
import gensim

#useful functions frequency vector functions 
from sklearn.feature_extraction.text import CountVectorizer #sklearn
from nltk.probability import FreqDist #nltk

#useful functions for tf-idf vectorization
from sklearn.feature_extraction.text import TfidfVectorizer #sklearn

#ngrams
from nltk.util import ngrams #nltk

ModuleNotFoundError: No module named 'gensim'

In [2]:
path = 'C:\\Users\\ritaf\\Documents\\ESTUDOS\\NOVA IMS\\3º Ano\\Text Mining\\Projeto\\data\\'

# Read the CSV file into a DataFrame
train_data = pd.read_csv(path + 'train.csv', index_col=-1)
test_data = pd.read_csv(path + 'test.csv', index_col=-1)

In [3]:
train_data.head()

,title,tag,artist,year,views,features,lyrics
id,,,,,,,
535805,Walk Away,rock,Tony Molina,2013,699,{},When you said you loved me\nDid you mean it th...
7519483,Gotta Make It Kid Naruto Rap,rap,Reece Lett,2021,4,{Sl!ck},Kid Naruto Rap\n[Hook]\nEverybody wants you to...
4892808,​this is what i asked for,pop,Elliot (DNK),2019,389,{},[Verse 1]\nPeople tell me I've changed\nI find...
1584150,Stealing Hearts,pop,Katie Armiger,2013,126,{},You've been warned about me\nDon't try to get ...
7639050,Get Ready,country,John Campbell Munro,1,2,{},[Verse 1]\nI can see the end is coming but I’v...


In [4]:
lyr = train_data['lyrics'].to_list()
lyr

["When you said you loved me\nDid you mean it then\nDid you change your mind one day\nOr was it all pretend\n\nCause it's over now\nSee you walk away\nSee you walk away\nSeems like nothing's gonna change",
 'Kid Naruto Rap\n[Hook]\nEverybody wants you to hurt\nEverybody wants you to cry with your face In the dirt\nGotta put in time, gotta work\nEverybody’s praying that I fall\nBut I gotta make it work\n\nEverybody wants you to hurt\nEverybody wants you to cry with your face In the dirt\nGotta put in time, gotta work\nEverybody’s praying that I fall\nBut I gotta make it\n\n[Verse]\nHoping it might work out one day\nWrong\nDo it yourself or you might as well fall\nGotta do the grind\nIf you never wanna fall\nBetter Aim for the top or you never won at all\nAs a kid I had nobody\nGrew up parentless\nHad to go and be funny\nHad to hide the pain some way\nOn my own path\nThat’s my ninja way\nHeaded for the top\nOn a day to day\nThey treat me like outcast\nGot Stay out the way\nGotta hide emo

In [11]:
def tokenized_frequency_vector(list_of_strings):
    '''Takes as input a list of strings of text and returns a list of dictionaries representing the frequency of each tokenized word'''

    # Create a tokenizer that only captures letters and numbers
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')

    # Initialize an empty list to store the frequency vectors
    frequency_vectors = []

    # Process each string in the input list
    for text in list_of_strings:
    
        # Tokenize the text data - converted to string to avoid issues with numbers or other data types
        tokens = tokenizer.tokenize(str(text))

        # Convert the tokens to lowercase
        tokens = [token.lower() for token in tokens]

        # Remove stop words
        #1. Create a set of stop words
        stop_words = set(nltk.corpus.stopwords.words('english')) #CODE HERE
    
        #remove stop words from the tokenized text
        tokens = [token for token in tokens if token not in stop_words] #CODE HERE

        # Create a frequency distribution for the tokenized text data
        fdist = FreqDist(tokens)

        # Append the frequency vector to the list of frequency vectors
        frequency_vectors.append(dict(fdist))

    return frequency_vectors

In [12]:
lyr_freq_vectors = tokenized_frequency_vector(lyr)
lyr_freq_vectors

[{'said': 1,
  'loved': 1,
  'mean': 1,
  'change': 2,
  'mind': 1,
  'one': 1,
  'day': 1,
  'pretend': 1,
  'cause': 1,
  'see': 2,
  'walk': 2,
  'away': 2,
  'seems': 1,
  'like': 1,
  'nothing': 1,
  'gonna': 1},
 {'kid': 2,
  'naruto': 1,
  'rap': 1,
  'hook': 3,
  'everybody': 22,
  'wants': 14,
  'hurt': 7,
  'cry': 7,
  'face': 7,
  'dirt': 7,
  'gotta': 24,
  'put': 7,
  'time': 7,
  'work': 12,
  'praying': 7,
  'fall': 9,
  'make': 8,
  'verse': 2,
  'hoping': 1,
  'might': 2,
  'one': 1,
  'day': 3,
  'wrong': 1,
  'well': 1,
  'grind': 1,
  'never': 2,
  'wanna': 1,
  'better': 1,
  'aim': 1,
  'top': 2,
  'nobody': 1,
  'grew': 1,
  'parentless': 1,
  'go': 1,
  'funny': 1,
  'hide': 2,
  'pain': 2,
  'way': 3,
  'path': 1,
  'ninja': 1,
  'headed': 1,
  'treat': 1,
  'like': 1,
  'outcast': 1,
  'got': 3,
  'stay': 1,
  'emotions': 1,
  'hidе': 1,
  'demon': 1,
  'fox': 1,
  'inside': 2,
  'keeps': 1,
  'driving': 1,
  'mе': 1,
  'insane': 1,
  '2': 1,
  'sl': 1,
  'ck'

In [14]:
#NÃO FUNCIONA PORQUE OS DADOS SÃO DEMASIADO GRANDES I THINK... PRECISO DOS PREPROCESSAMENTOS DO CADETE
lyr_freq_df_1 = pd.DataFrame(lyr_freq_vectors)

# Sort the columns by their sums in descending order
sorted_cols = lyr_freq_df_1.sum().sort_values(ascending=False).index

# Reorder the columns of the DataFrame
lyr_freq_df_1 = lyr_freq_df_1[sorted_cols]

#this creates a sparse matrix
lyr_freq_df_1

MemoryError: Unable to allocate 235. GiB for an array with shape (134967, 233443) and data type object